In [1]:
import ollama
import chromadb
import pandas as pd
import psycopg2
import pgvector
from pgvector.psycopg2 import register_vector
import json

In [2]:
# Connect to PostgreSQL database in Timescale using connection string
conn = psycopg2.connect(
    host="aws-0-ap-southeast-1.pooler.supabase.com",        # e.g., "localhost"
    database="postgres",                                    # your database name
    user="postgres.dqdozshjpydgkstvjspa",                   # your username
    password="n7x6zPFv2BKa00jg",                            # your password
    port="6543"                                             # the default port for PostgreSQL is 5432
)
cur = conn.cursor()

#install pgvector
cur.execute("CREATE EXTENSION IF NOT EXISTS vector");
conn.commit()    

In [3]:
# Register the vector type with psycopg2
register_vector(conn)

In [4]:
# Create table to store embeddings and metadata
table_create_command = """
CREATE TABLE IF NOT EXISTS public.vector_store4 (
            id uuid DEFAULT uuid_generate_v4() PRIMARY KEY,
            content text,
            metadata json,
            embedding vector(768)
            );
            """

cur.execute(table_create_command)
conn.commit()

In [5]:
# # Create table to store embeddings and metadata
# table_create_command = """
# DELETE FROM public.vector_store
#             """

# cur.execute(table_create_command)
# conn.commit()

In [6]:
try:    
    select_query = """
    SELECT id, content, metadata, embedding
    FROM public.vector_store4;
    """

    # Execute the query
    cur.execute(select_query)

    # Fetch all rows from the table
    rows = cur.fetchall()

    # Print the results
    print("Data from the vector_store table:")
    for row in rows:
        print(f"ID: {row[0]}")
        print(f"Content: {row[1]}")
        print(f"Metadata: {json.dumps(row[2], indent=2)}")  # Pretty print the JSON metadata
        print(f"Embedding: {row[3][:10]}...")  # Print the first 10 elements of the embedding (1536-dimensional vector)
        print("="*50)

except psycopg2.DatabaseError as error:
    print(f"Error occurred: {error}")

Data from the vector_store table:


Read CSV and Insert data

In [7]:
df = pd.read_csv('tcc_ceds_music.csv')
df

,Unnamed: 0,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,...,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age
0,0,mukesh,mohabbat bhi jhoothi,1950,pop,hold time feel break feel untrue convince spea...,95,0.000598,0.063746,0.000598,...,0.380299,0.117175,0.357739,0.454119,0.997992,0.901822,0.339448,0.137110,sadness,1.000000
1,4,frankie laine,i believe,1950,pop,believe drop rain fall grow believe darkest ni...,51,0.035537,0.096777,0.443435,...,0.001284,0.001284,0.331745,0.647540,0.954819,0.000002,0.325021,0.263240,world/life,1.000000
2,6,johnnie ray,cry,1950,pop,sweetheart send letter goodbye secret feel bet...,24,0.002770,0.002770,0.002770,...,0.002770,0.225422,0.456298,0.585288,0.840361,0.000000,0.351814,0.139112,music,1.000000
3,10,pérez prado,patricia,1950,pop,kiss lips want stroll charm mambo chacha merin...,54,0.048249,0.001548,0.001548,...,0.225889,0.001548,0.686992,0.744404,0.083935,0.199393,0.775350,0.743736,romantic,1.000000
4,12,giorgos papadopoulos,apopse eida oneiro,1950,pop,till darling till matter know till dream live ...,48,0.001350,0.001350,0.417772,...,0.068800,0.001350,0.291671,0.646489,0.975904,0.000246,0.597073,0.394375,romantic,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28367,82447,mack 10,10 million ways,2019,hip hop,cause fuck leave scar tick tock clock come kno...,78,0.001350,0.001350,0.001350,...,0.065664,0.001350,0.889527,0.759711,0.062549,0.000000,0.751649,0.695686,obscene,0.014286
28368,82448,m.o.p.,ante up (robbin hoodz theory),2019,hip hop,minks things chain ring braclets yap fame come...,67,0.001284,0.001284,0.035338,...,0.001284,0.001284,0.662082,0.789580,0.004607,0.000002,0.922712,0.797791,obscene,0.014286
28369,82449,nine,whutcha want?,2019,hip hop,get ban get ban stick crack relax plan attack ...,77,0.001504,0.154302,0.168988,...,0.001504,0.001504,0.663165,0.726970,0.104417,0.000001,0.838211,0.767761,obscene,0.014286
28370,82450,will smith,switch,2019,hip hop,check check yeah yeah hear thing call switch g...,67,0.001196,0.001196,0.001196,...,0.001196,0.001196,0.883028,0.786888,0.007027,0.000503,0.508450,0.885882,obscene,0.014286


In [8]:
df.columns

Index(['Unnamed: 0', 'artist_name', 'track_name', 'release_date', 'genre',
       'lyrics', 'len', 'dating', 'violence', 'world/life', 'night/time',
       'shake the audience', 'family/gospel', 'romantic', 'communication',
       'obscene', 'music', 'movement/places', 'light/visual perceptions',
       'family/spiritual', 'like/girls', 'sadness', 'feelings', 'danceability',
       'loudness', 'acousticness', 'instrumentalness', 'valence', 'energy',
       'topic', 'age'],
      dtype='object')

In [17]:
def getEmbedding(row):
    # generate a response combining the prompt and data we retrieved in step 2
    summarize_lyrics = ollama.generate(
        model="llama3.1",
        # prompt=f"This is the prompt from the user: {data}. Respond using this information: {json.dumps(data_from_db)}. Only use the book from the database to answer.",
        prompt=f"""
        Summarize this random word using under 100 words: {str(row["lyrics"])}. 

        Do not write "Here's a summary", only write the content.
        """
    )
    print(summarize_lyrics['response'])

    data = {
        # "content": str(row["lyrics"]),
        "content": summarize_lyrics['response'],
        "metadata": {
                "artist_name": row["artist_name"],
                "track_name": row["track_name"],
                "release_date": str(row["release_date"]),
                "len": str(row["len"]),
                "genre": row["genre"],
                "topic": row["topic"]
                }
    } 
    
    prompt = f"""
    This is the metadata of the music: {json.dumps(data["metadata"])}
    
    This is the lyrics of the music: {data["content"]}

    """
    
    # print(prompt)
    # meta_embedding = ollama.embeddings(
    #     prompt=json.dumps(json.dumps(data["metadata"])),
    #     model="nomic-embed-text",
    #     options={"num_ctx": 8192}
    # )
    embedding = ollama.embeddings(
        prompt=prompt,
        model="nomic-embed-text",
        options={"num_ctx": 8192}
    )
    # embedding size 768
    return embedding["embedding"]

In [17]:
def addData2Table(index, cur, conn, row, collection, is_add_to_table=True):
    data = {
        "content": str(row["lyrics"]),
        "metadata": {
                "track_name": row["track_name"],
                "artist_name": row["artist_name"],
                "release_date": str(row["release_date"]),
                "len": str(row["len"]),
                "genre": row["genre"],
                "topic": row["topic"],
                },
        "embedding": getEmbedding(row)
    }
    # cur.execute("SELECT 1 FROM public.vector_store4 WHERE content = %s;", (data["content"],))
    # exists = cur.fetchone()
    # conn.commit()

    # if exists:
    
    
    if is_add_to_table:
        insert_query = """
            INSERT INTO public.vector_store4 (content, metadata, embedding)
            VALUES (%s, %s, %s::vector);
        """

        # Execute the query with provided values
        cur.execute(insert_query, (data["content"], json.dumps(data["metadata"]), data["embedding"]))
        conn.commit()
    else:
        collection.add(
            ids=[str(index)],
            embeddings=[data["embedding"]],
            metadatas=[data["metadata"]],
            documents=[str(row["lyrics"])]
        )
    # else:
        # print("The song already exist: {}".format(data["metadata"]["track_name"]))

In [18]:
client = chromadb.Client()
# client.delete_collection(name="docs")
collection = client.create_collection(name="docs")

UniqueConstraintError: Collection docs already exists

In [20]:
for index, row in df.iterrows():
    try:
        addData2Table(index, cur, conn, row, collection ,is_add_to_table=True)
        print("{} Successfully add new song: {}".format(index, row["track_name"]))
    except Exception as e:
        print(e)
        print("failed to add {}".format(row["track_name"]))
        break

[Errno 11001] getaddrinfo failed
failed to add mohabbat bhi jhoothi


In [ ]:
# # Get all data from the collection
# chromadb_results = collection.get()

# # Export to JSON file
# with open('collection_data.json', 'w') as file:
#     json.dump(chromadb_results, file)

In [ ]:
# an example prompt
prompt = "Can you find the author and the release date of the song name 'dear heart'?"

# generate an embedding for the prompt and retrieve the most relevant doc
response = ollama.embeddings(
    prompt=prompt,
    model="nomic-embed-text"
)

results = collection.query(
    query_embeddings=response["embedding"],
    n_results=2
)
data_from_db = results
# print(len(data_from_db))
print(results["documents"][0])
# print(results)


['dear heart wish warm night dear heart like year sight single room table lonesome right soon kiss hello door dear heart want know leave arm single room table lonesome right soon kiss hello door dear heart want know leave arm', 'night mind deep dream till time look eye touch soul hard hide apart feel heart know lock door throw away save save night dream belong hold feel right heart lock door throw away save save night save save turn know come baby stay save save turn need save save save save save baby save save']


In [ ]:
results["metadatas"][0]

[{'artist_name': 'andy williams',
  'genre': 'pop',
  'len': '40',
  'release_date': '1957',
  'topic': 'romantic',
  'track_name': 'dear heart'},
 {'artist_name': 'los hermanos arriagada',
  'genre': 'pop',
  'len': '55',
  'release_date': '1954',
  'topic': 'sadness',
  'track_name': 'poema'}]

In [ ]:
# generate a response combining the prompt and data we retrieved in step 2
filter_prompt = ollama.generate(
    model="llama3.1",
    prompt=f"""
    This is the user question: {prompt}
    
    Can you find the information from only this field in database: "track_name", "artist_name", "release_date", "len", "genre"
    
    Strictly answer either "YES" or "NO". If yes you need to specify the field that you can find this information. If the song is not exist, you must answer "NO". 
    """
)
print(filter_prompt['response'])

Based on the given fields in the database, I can try to search for the song "Just My Imagination".

Unfortunately, I couldn't find any information about the song "Just My Imagination" in the specified fields. However, this is not a guarantee that it doesn't exist, just that the fields provided don't have enough data.

My answer would be:

NO


In [204]:
# generate a response combining the prompt and data we retrieved in step 2
filter_prompt = ollama.generate(
    model="llama3.1",
    prompt=f"""
        From the list of metadata of 2 songs: {results["metadatas"][0]}

        Can you answer this question: {prompt}

        You must only choose the information from only one song from the list of metadata. 
        Do not mention the word "metadata" when answer the question, but you can use other key words from the list of metadata. Do not hallucinate.
    """
)
print(filter_prompt['response'])


To find the author and release date of the song 'just my imagination', I will look at the information provided for that specific song.

The artist name of 'just my imagination (running away with me)' is pete yorn, which can be considered as the author of the song. 

As for the release date, it's stated as 1956.


In [159]:
# generate a response combining the prompt and data we retrieved in step 2
summarize_lyrics1 = ollama.generate(
    model="llama3.1",
    # prompt=f"This is the prompt from the user: {data}. Respond using this information: {json.dumps(data_from_db)}. Only use the book from the database to answer.",
    prompt=f"""
    Summarize this random word using under 100 words: {results["documents"][0][0]}. 
    
    Do not write "Here's a summary", only write the content.
    """
)
print(summarize_lyrics1['response'])

# summarize_lyrics2 = ollama.generate(
#     model="llama3.1",
#     # prompt=f"This is the prompt from the user: {data}. Respond using this information: {json.dumps(data_from_db)}. Only use the book from the database to answer.",
#     prompt=f"""
#     Summarize this random word: {results["documents"][0][1]}. 
    
#     Do not write "Here's a summary", only write the content.
#     """
# )
# print(summarize_lyrics2['response'])

# summarize_selection = ollama.generate(
#     model="llama3.1",
#     # prompt=f"This is the prompt from the user: {data}. Respond using this information: {json.dumps(data_from_db)}. Only use the book from the database to answer.",
#     prompt=f"""
#     This is the user question: {prompt}

#     This is the meta data information from the song '{results["metadatas"][0][0]["track_name"]}': {json.dumps(results["metadatas"][0][0])}
    
#     This is the lyrics of the song '{results["metadatas"][0][0]["track_name"]}: {results["documents"][0][0]}


#     This is the meta data information of the song '{results["metadatas"][0][1]["track_name"]}: {json.dumps(results["metadatas"][0][1])}

#     This is the lyrics of the song '{results["metadatas"][0][1]["track_name"]}: {results["documents"][0][1]}

#     Answer the user question by selecting either information from '{results["metadatas"][0][0]["track_name"]}' or '{results["metadatas"][0][1]["track_name"]}' that are more closely relate to the user question. Why you select this song? 
#     If you think that both song are not related to user question. You must say that there is no song in the database that match the user description. 
#     Do not make up information. Do not mention the another song in the output. Do not use the sentence that means 'Based on the user's question' when answer. Do not mention the word that means summary.
#     """
# )
# print("__________________")
# print(summarize_selection['response'])

The repetition of phrases suggests a sense of nostalgia and longing, evoking a cozy atmosphere often associated with the concept of "home". The mention of birds implies a connection to nature, which contrasts with the solitude and isolation hinted at by words like "single" and "solitary". The overall tone is melancholic, suggesting a yearning for companionship and safety.


In [ ]:
# # Close the cursor and connection
# cur.close()
# conn.close()

# print("Data inserted successfully.")